In [1]:
import os
os.getpid()

10668

In [2]:
import pandas as pd
import numpy as np
import re

In [3]:
import pathlib
dol_dir = pathlib.Path('.').parent.absolute()
dol_locations_file = (dol_dir / 'ALL_FinalCrossing - Sheet1.csv').as_posix()
dol_locations_file

'C:/Users/bill.WSW011/Documents/Python/Dolphin-Vocalization/ALL_FinalCrossing - Sheet1.csv'

In [4]:
dol = pd.read_csv(dol_locations_file, header=[0, 1])
dol.head()

Unnamed: 0_level_0 Unnamed: 1_level_0 MN                        0 ...    SH  \
                DATE               TIME  A  C    L  M  N    S  T  0 ...     S   
0           2-Feb-13            8:00:00  A  C    L  M  N  NaN  T  0 ...   NaN   
1           2-Feb-13            8:00:12  A  C    L  M  N    S  T  0 ...   NaN   
2           2-Feb-13            8:00:13  A  C  NaN  M  N    S  T  0 ...   NaN   
3           2-Feb-13            8:00:48  A  C    L  M  N    S  T  0 ...   NaN   
4           2-Feb-13            8:01:00  A  C    L  M  N    S  T  0 ...   NaN   

         0   G4                                
     T 0.2    A    C    L    M    N    S    T  
0  NaN   0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1  NaN   0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2  NaN   0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3  NaN   0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4  NaN   0  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 65 columns]

In [5]:
dol.shape

(5202, 65)

In [6]:
dol.columns

MultiIndex(levels=[['0', '1', 'DD', 'G1', 'G2', 'G3', 'G4', 'MN', 'NH', 'SH', 'Unnamed: 0_level_0', 'Unnamed: 17_level_0', 'Unnamed: 1_level_0', 'Unnamed: 33_level_0', 'Unnamed: 49_level_0'], ['0', '0.1', '0.2', '1', 'A', 'C', 'DATE', 'L', 'M', 'N', 'S', 'T', 'TIME', 'Unnamed: 17_level_1', 'Unnamed: 33_level_1', 'Unnamed: 49_level_1']],
           labels=[[10, 12, 7, 7, 7, 7, 7, 7, 7, 0, 3, 3, 3, 3, 3, 3, 3, 11, 8, 8, 8, 8, 8, 8, 8, 1, 4, 4, 4, 4, 4, 4, 4, 13, 2, 2, 2, 2, 2, 2, 2, 0, 5, 5, 5, 5, 5, 5, 5, 14, 9, 9, 9, 9, 9, 9, 9, 0, 6, 6, 6, 6, 6, 6, 6], [6, 12, 4, 5, 7, 8, 9, 10, 11, 0, 4, 5, 7, 8, 9, 10, 11, 13, 4, 5, 7, 8, 9, 10, 11, 3, 4, 5, 7, 8, 9, 10, 11, 14, 4, 5, 7, 8, 9, 10, 11, 1, 4, 5, 7, 8, 9, 10, 11, 15, 4, 5, 7, 8, 9, 10, 11, 2, 4, 5, 7, 8, 9, 10, 11]])

In [7]:
dol_ids = ['A', 'C', 'L', 'M', 'N', 'S', 'T']
tanks_and_gates = 'MN								G1								NH								G2								DD								G3								SH								G4'

In [8]:
tanks_and_gates = [i for i in tanks_and_gates.split() if i]
tanks_and_gates

['MN', 'G1', 'NH', 'G2', 'DD', 'G3', 'SH', 'G4']

In [9]:
tank_ids = [t for t in tanks_and_gates if not t.startswith('G')]
gate_ids = [g for g in tanks_and_gates if g.startswith('G')]
dol_ids, tank_ids, gate_ids

(['A', 'C', 'L', 'M', 'N', 'S', 'T'],
 ['MN', 'NH', 'DD', 'SH'],
 ['G1', 'G2', 'G3', 'G4'])

In [10]:
from collections import namedtuple

In [11]:
GateNeighbor = namedtuple('GateNeighbor', ['to_left', 'to_right'])

In [12]:
tg_len = len(tanks_and_gates)

In [13]:
gate_neighbors = {}
for i in range(1, tg_len):
    item = tanks_and_gates[i]
    if item.startswith('G'):
        left = tanks_and_gates[i - 1]
        if i == tg_len - 1:
            right = tanks_and_gates[0]
        else:
            right = tanks_and_gates[i + 1]
        gate_neighbors[item] = GateNeighbor(to_left=left, to_right=right)

In [14]:
gate_neighbors

{'G1': GateNeighbor(to_left='MN', to_right='NH'),
 'G2': GateNeighbor(to_left='NH', to_right='DD'),
 'G3': GateNeighbor(to_left='DD', to_right='SH'),
 'G4': GateNeighbor(to_left='SH', to_right='MN')}

In [15]:
new_cols = ['date', 'time'] + [tg + '-' + dol for tg in tanks_and_gates
                              for dol in  ['closed'] + dol_ids][1:]

In [16]:
len(new_cols)

65

In [17]:
dol2 = pd.read_csv(dol_locations_file,
                   names=new_cols, skiprows=2, parse_dates={'date_time': ['date', 'time']}, keep_date_col=[True, True])
dol2.fillna('', inplace=True)
dol2.head()

,date_time,date,time,MN-A,MN-C,MN-L,MN-M,MN-N,MN-S,MN-T,...,SH-S,SH-T,G4-closed,G4-A,G4-C,G4-L,G4-M,G4-N,G4-S,G4-T
0,2-Feb-13 8:00:00,2-Feb-13,8:00:00,A,C,L,M,N,,T,...,,,0,,,,,,,
1,2-Feb-13 8:00:12,2-Feb-13,8:00:12,A,C,L,M,N,S,T,...,,,0,,,,,,,
2,2-Feb-13 8:00:13,2-Feb-13,8:00:13,A,C,,M,N,S,T,...,,,0,,,,,,,
3,2-Feb-13 8:00:48,2-Feb-13,8:00:48,A,C,L,M,N,S,T,...,,,0,,,,,,,
4,2-Feb-13 8:01:00,2-Feb-13,8:01:00,A,C,L,M,N,S,T,...,,,0,,,,,,,


In [18]:
dol2.tail()

,date_time,date,time,MN-A,MN-C,MN-L,MN-M,MN-N,MN-S,MN-T,...,SH-S,SH-T,G4-closed,G4-A,G4-C,G4-L,G4-M,G4-N,G4-S,G4-T
5197,4-Feb-13 16:59:00,4-Feb-13,16:59:00,A,,L,M,N,,T,...,,,0,,,,,,,
5198,4-Feb-13 16:59:27,4-Feb-13,16:59:27,A,,L,M,N,,T,...,,,0,,,,,,,
5199,4-Feb-13 16:59:42,4-Feb-13,16:59:42,A,,L,M,N,,T,...,,,0,,,,,,,
5200,4-Feb-13 16:59:54,4-Feb-13,16:59:54,A,,L,M,N,,T,...,,,0,,,,,,,
5201,4-Feb-13 17:00:00,4-Feb-13,17:00:00,A,,L,M,N,,T,...,,,0,,,,,,,


In [19]:
dol2.dtypes[:2]

date_time    object
date         object
dtype: object

In [20]:
no_dates = dol2[dol2['date'] == ''].index
no_dates

Int64Index([663, 1056, 1406, 2414, 2953, 3496, 4173, 4662], dtype='int64')

In [21]:
dol2.drop(no_dates, inplace=True)

In [22]:
date_strings = dol2['date'] + ' ' + dol2['time']
date_strings.shape

(5194,)

In [23]:
dol2[dol2['date'].str.strip() == '']

,date_time,date,time,MN-A,MN-C,MN-L,MN-M,MN-N,MN-S,MN-T,...,SH-S,SH-T,G4-closed,G4-A,G4-C,G4-L,G4-M,G4-N,G4-S,G4-T
2793,13:20:04,,13:20:04,,,,M,,S,,...,,,0,,,,,,,


In [24]:
# .loc[2793]

dol2.at[2793, 'date'] = dol2.at[2792, 'date']
dol2.at[2793, 'date_time'] = dol2.at[2792, 'date_time']

In [25]:
dol2.head()

,date_time,date,time,MN-A,MN-C,MN-L,MN-M,MN-N,MN-S,MN-T,...,SH-S,SH-T,G4-closed,G4-A,G4-C,G4-L,G4-M,G4-N,G4-S,G4-T
0,2-Feb-13 8:00:00,2-Feb-13,8:00:00,A,C,L,M,N,,T,...,,,0,,,,,,,
1,2-Feb-13 8:00:12,2-Feb-13,8:00:12,A,C,L,M,N,S,T,...,,,0,,,,,,,
2,2-Feb-13 8:00:13,2-Feb-13,8:00:13,A,C,,M,N,S,T,...,,,0,,,,,,,
3,2-Feb-13 8:00:48,2-Feb-13,8:00:48,A,C,L,M,N,S,T,...,,,0,,,,,,,
4,2-Feb-13 8:01:00,2-Feb-13,8:01:00,A,C,L,M,N,S,T,...,,,0,,,,,,,


In [26]:
for g in range(1, 5):
    print(dol2[f'G{g}-closed'].value_counts())

0    3081
1    2113
Name: G1-closed, dtype: int64
0    3317
1    1877
Name: G2-closed, dtype: int64
0    5192
1       2
Name: G3-closed, dtype: int64
0    5194
Name: G4-closed, dtype: int64


In [27]:
def fill_positions(df, dolphin_list, gate_neighbors):
    first_col = df.columns[0]
    for dolphin in dolphin_list:
        # First gather positions; note that rows without a dolphin in a tank column
        # will report a position of the first 2 letters of the first column
        moving = {}
        moving['right'] = df == dolphin + '>'
        moving['left'] = df == '<' + dolphin
        dol_pos = ((df == dolphin) | moving['right'] | moving['left']).idxmax(axis=1)
        df[dolphin + '_position'] = dol_pos.str.slice(stop=-2)
        # Determine which direction they're moving if in a gate
        for direction in ['left', 'right']:
            idx_max = moving[direction].idxmax(axis=1)
            idx_max_good = idx_max[idx_max != first_col]
            idx_max_good = idx_max_good.str.slice(stop=-2)
            if direction == 'right':
                moving_to = idx_max_good.apply(lambda x: gate_neighbors[x].to_right)
            else:
                moving_to = idx_max_good.apply(lambda x: gate_neighbors[x].to_left)
            
            df[dolphin + '_in_gate_moving_toward'] = moving_to
    df.fillna('', inplace=True)
        

In [28]:
fill_positions(dol2, dol_ids, gate_neighbors)

In [29]:
dol2.head(25)

,date_time,date,time,MN-A,MN-C,MN-L,MN-M,MN-N,MN-S,MN-T,...,L_position,L_in_gate_moving_toward,M_position,M_in_gate_moving_toward,N_position,N_in_gate_moving_toward,S_position,S_in_gate_moving_toward,T_position,T_in_gate_moving_toward
0,2-Feb-13 8:00:00,2-Feb-13,8:00:00,A,C,L,M,N,,T,...,MN,,MN,,MN,,NH,,MN,
1,2-Feb-13 8:00:12,2-Feb-13,8:00:12,A,C,L,M,N,S,T,...,MN,,MN,,MN,,MN,,MN,
2,2-Feb-13 8:00:13,2-Feb-13,8:00:13,A,C,,M,N,S,T,...,SH,,MN,,MN,,MN,,MN,
3,2-Feb-13 8:00:48,2-Feb-13,8:00:48,A,C,L,M,N,S,T,...,MN,,MN,,MN,,MN,,MN,
4,2-Feb-13 8:01:00,2-Feb-13,8:01:00,A,C,L,M,N,S,T,...,MN,,MN,,MN,,MN,,MN,
5,2-Feb-13 8:01:01,2-Feb-13,8:01:01,A,C,,M,N,S,T,...,NH,,MN,,MN,,MN,,MN,
6,2-Feb-13 8:01:45,2-Feb-13,8:01:45,A,C,,M,N,S,T,...,G2,DD,MN,,MN,,MN,,MN,
7,2-Feb-13 8:01:52,2-Feb-13,8:01:52,A,C,,M,N,S,T,...,NH,,MN,,MN,,MN,,MN,
8,2-Feb-13 8:02:00,2-Feb-13,8:02:00,A,C,,M,N,S,T,...,NH,,MN,,MN,,MN,,MN,
9,2-Feb-13 8:03:00,2-Feb-13,8:03:00,A,C,,M,N,S,T,...,NH,,MN,,MN,,MN,,MN,


In [30]:
def list_no_none(items):
    return [re.sub(r'[<>]', '', i) for i in items if i and type(i) == str]

In [31]:
def fill_dolphins(df, position_list):
    for pos in position_list:
        # Columns matching the positions
        pos_cols = [c for c in df.columns if c.startswith(pos)]
        df[pos + '_contains'] = df[pos_cols].apply(list_no_none, axis=1)


In [32]:
fill_dolphins(dol2, tanks_and_gates)

In [33]:
dol2.replace(to_replace='date_ti', value='', inplace=True)

In [34]:
dol2.head()

,date_time,date,time,MN-A,MN-C,MN-L,MN-M,MN-N,MN-S,MN-T,...,T_position,T_in_gate_moving_toward,MN_contains,G1_contains,NH_contains,G2_contains,DD_contains,G3_contains,SH_contains,G4_contains
0,2-Feb-13 8:00:00,2-Feb-13,8:00:00,A,C,L,M,N,,T,...,MN,,"[A, C, L, M, N, T]",[],[S],[],[],[],[],[]
1,2-Feb-13 8:00:12,2-Feb-13,8:00:12,A,C,L,M,N,S,T,...,MN,,"[A, C, L, M, N, S, T]",[],[],[],[],[],[],[]
2,2-Feb-13 8:00:13,2-Feb-13,8:00:13,A,C,,M,N,S,T,...,MN,,"[A, C, M, N, S, T]",[],[],[],[],[],[L],[]
3,2-Feb-13 8:00:48,2-Feb-13,8:00:48,A,C,L,M,N,S,T,...,MN,,"[A, C, L, M, N, S, T]",[],[],[],[],[],[],[]
4,2-Feb-13 8:01:00,2-Feb-13,8:01:00,A,C,L,M,N,S,T,...,MN,,"[A, C, L, M, N, S, T]",[],[],[],[],[],[],[]


In [35]:
def find_lonely_dolphin(df, dolphin_id):
    # Find the dolphin
    dol_pos = df[f'{dolphin_id}_position']
    # Filter no location
    in_tank = dol_pos != ''
    df_with_dol = df[in_tank]
    # Make column names to check
    tank_cols = dol_pos + '_contains'
    # Get values which include the dolphin question
    dol_friends = df_with_dol.lookup(df_with_dol.index, tank_cols[in_tank])
    dol_alone = [dp[:2] if len(f) == 1 else '' for f, dp in zip(dol_friends, dol_pos[in_tank])]
    new_col = dolphin_id + '_alone'
    df.loc[in_tank, new_col] = dol_alone
    df[new_col].fillna('', inplace=True)

In [36]:
for d in dol_ids:
    find_lonely_dolphin(dol2, d)

In [37]:
dol_ids[-1]

'T'

In [38]:
dol2[dol_ids[-1] + '_alone'].value_counts()

      4793
DD     160
NH      89
SH      65
G2      44
G3      22
G1      20
G4       1
Name: T_alone, dtype: int64

In [39]:
wanted_cols = ['date_time'] + sorted([c for c in dol2.columns if '_' in c and c != 'date_time'])
# wanted_cols

In [40]:
our_dol2 = dol2[wanted_cols]

In [41]:
our_dol2.head(25)

,date_time,A_alone,A_in_gate_moving_toward,A_position,C_alone,C_in_gate_moving_toward,C_position,DD_contains,G1_contains,G2_contains,...,N_alone,N_in_gate_moving_toward,N_position,SH_contains,S_alone,S_in_gate_moving_toward,S_position,T_alone,T_in_gate_moving_toward,T_position
0,2-Feb-13 8:00:00,,,MN,,,MN,[],[],[],...,,,MN,[],NH,,NH,,,MN
1,2-Feb-13 8:00:12,,,MN,,,MN,[],[],[],...,,,MN,[],,,MN,,,MN
2,2-Feb-13 8:00:13,,,MN,,,MN,[],[],[],...,,,MN,[L],,,MN,,,MN
3,2-Feb-13 8:00:48,,,MN,,,MN,[],[],[],...,,,MN,[],,,MN,,,MN
4,2-Feb-13 8:01:00,,,MN,,,MN,[],[],[],...,,,MN,[],,,MN,,,MN
5,2-Feb-13 8:01:01,,,MN,,,MN,[],[],[],...,,,MN,[],,,MN,,,MN
6,2-Feb-13 8:01:45,,,MN,,,MN,[],[],[L],...,,,MN,[],,,MN,,,MN
7,2-Feb-13 8:01:52,,,MN,,,MN,[],[],[],...,,,MN,[],,,MN,,,MN
8,2-Feb-13 8:02:00,,,MN,,,MN,[],[],[],...,,,MN,[],,,MN,,,MN
9,2-Feb-13 8:03:00,,,MN,,,MN,[],[],[],...,,,MN,[],,,MN,,,MN


In [42]:
our_dol2.to_csv('processed_dolphin_locations.csv')

In [43]:
dol3 = pd.read_csv('processed_dolphin_locations.csv', index_col=0)
dol3.fillna('', inplace=True)

In [44]:
(dol3 != our_dol2).sum().sum()

41552

In [45]:
dol2['DD_contains'][14], dol3['DD_contains'][14]

(['A', 'T'], "['A', 'T']")

In [46]:
eval(dol3['DD_contains'][14])

['A', 'T']

In [47]:
contain_cols = [c for c in our_dol2.columns if c.endswith('_contains')]

In [48]:
contain_cols

['DD_contains',
 'G1_contains',
 'G2_contains',
 'G3_contains',
 'G4_contains',
 'MN_contains',
 'NH_contains',
 'SH_contains']

In [49]:
contains_converters = {c: eval for c in ['DD_contains',
 'G1_contains',
 'G2_contains',
 'G3_contains',
 'G4_contains',
 'MN_contains',
 'NH_contains',
 'SH_contains']}

In [50]:
dol3 = pd.read_csv('processed_dolphin_locations.csv', index_col=0, converters=contains_converters)
dol3.fillna('', inplace=True)

In [51]:
(dol3 != our_dol2).sum().sum()

0